In [1]:
import os, sys, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import SGDClassifier
# refer to https://tianchi.aliyun.com/learn/live.html?spm=5176.11510297.4851204.1.19a93c60417WIG&classroomId=263#classroomId=263&postsId=23463

In [3]:
# 数据加载
dir_file = 'data'
dfoff = pd.read_csv(os.path.join(dir_file, 'cf_offline_stage1_train.csv'))
dftest = pd.read_csv(os.path.join(dir_file, 'ccf_offline_stage1_test_revised.csv'))
dfon = pd.read_csv(os.path.join(dir_file, 'ccf_online_stage1_train.csv'))
print('data read end.')

In [3]:
# 1. 将满xx减yy类型(`xx:yy`)的券变成折扣率 : `1 - yy/xx`，同时建立折扣券相关的特征 
# `discount_rate, discount_man, discount_jian, discount_type`
# 2. 将距离 `str` 转为 `int`
# convert Discount_rate and Distance

def getDiscountType(row):
  if pd.isnull(row):
    return np.nan
  elif ':' in row:
    return 1
  else:
    return 0

def convertRate(row):
  if pd.isnull(row):
    return 1.0
  elif ':' in str(row):
    rows = str(row).split(':')
    return 1.0 - float(rows[1])/float(rows[0])
  else:
    return float(row)

def getDiscountMan(row):
  if ':' in str(row):
    rows = str(row).split(':')
    return int(rows[0])
  else:
    return 0

def getDiscountJian(row):
  if ':' in str(row):
    rows = str(row).split(':')
    return int(rows[1])
  else:
    return 0
print('tool is ok')

In [4]:
def processData(df):
  df['discount_rate'] = df['Discount_rate'].apply(convertRate)
  df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
  df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
  df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
  print(df['discount_rate'].unique())
  df['distance'] = df['Distance'].fillna(-1).astype(int)
  return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [ ]:
# date_received = dfoff['Date_received'].unique()
# date_received = sorted(date_received[pd.notnull(date_received)])

# date_buy = dfoff['Date'].unique()
# date_buy = sorted(date_buy[pd.notnull(date_buy)])
# date_buy = sorted(dfoff[dfoff['Date'].notnull()]['Date'])

In [ ]:
couponbydate = dfoff[dfoff['Date_received'].notnull()][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
couponbydate.columns = ['Date_received','count']
buybydate = dfoff[(dfoff['Date'].notnull()) & (dfoff['Date_received'].notnull())][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
buybydate.columns = ['Date_received','count']
 
print("end")

In [4]:
def getWeekday(row):
    if row == 'nan':
        return np.nan
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1
 
dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)
 
# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
dftest['weekday_type'] = dftest['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
 
# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
tmpdf = pd.get_dummies(dfoff['weekday'])
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf
 
tmpdf = pd.get_dummies(dftest['weekday'])
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf
 
 
def label(row):
    if pd.isnull(row['Date_received']):
        return -1
    if pd.notnull(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
dfoff['label'] = dfoff.apply(label, axis = 1)
 
print("end")

In [ ]:
print("-----data split------")
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < 20160516)].copy()
valid = df[(df['Date_received'] >= 20160516) & (df['Date_received'] <= 20160615)].copy()
print("end")

In [4]:
# feature
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','distance', 'weekday', 'weekday_type'] + weekdaycols
print("----train-----")
model = SGDClassifier(#lambda:
    loss='log',  # 损失函数
    penalty='elasticnet',  # 结合L1和L2范数
    fit_intercept=True,  # 是痘存在截距
    max_iter=100, # 循环次数
    shuffle=True, # 打乱顺序
    alpha=0.01,  # 结合L1和L2范数的参数
    l1_ratio=0.01, # 结合L1和L2范数的参数
    n_jobs=1, # cpu个数
    class_weight=None
)
model.fit(train[original_feature], train['label'])

In [ ]:
# #### 预测以及结果评价
print(model.score(valid[original_feature], valid['label']))


In [4]:
print("---save model---")
with open('1_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('1_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
# test prediction for submission
y_test_pred = model.predict_proba(dftest[original_feature])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
dftest1['label'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head()